Lets do few shot inference against our EHR FHIR PostGre SQL database



1.   Load and test Open AI access
2.   Load and test EHR PostgreSQL database access
3.   Load FAISS









# 0. Mount drive and define paths

# Installs

In [ ]:
# --- 0) Mount Drive and set your variables ---
from google.colab import drive
drive.mount('/content/drive')

DEV_PATH = "/content/drive/MyDrive/210_Capstone/210_Factory/210_dev"
FAISS_DB_PATH = DEV_PATH + "/vectorstores/medintellagent_faiss_v1"
POSTGRES_DB_PATH = DEV_PATH + "/synthea_ehr_backup.sql"  # (dump file; not used in this snippet)
LLM_MODEL = "gpt-4o-mini"
EMBEDDING_MODEL = "text-embedding-3-large"



Mounted at /content/drive


In [ ]:
%%capture
# --- 1) Install libraries (once per runtime) ---
!pip -q install --upgrade openai langchain langchain-community langchain-openai faiss-cpu


In [ ]:
# --- 2) Load your OpenAI key (recommended: Colab "Secrets" → OPENAI_API_KEY) ---
import os
try:
    from google.colab import userdata
    key = userdata.get('OPENAI_API_KEY')
    if key: os.environ['OPENAI_API_KEY'] = key
except Exception:
    pass

if not os.environ.get("OPENAI_API_KEY"):
    import getpass
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPENAI_API_KEY: ")

# Initialize OpenAI client
from openai import OpenAI
client = OpenAI()


In [ ]:
# --- 3) Load  FAISS vector store (must use same embedding model used to build it) ---
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vs = FAISS.load_local(FAISS_DB_PATH, embeddings, allow_dangerous_deserialization=True)

print("FAISS loaded. Example count:", len(vs.docstore._dict))


FAISS loaded. Example count: 80


In [ ]:
# --- Prompt building helpers ---

# Keep this aligned to your actual DB schema + rules
PREFIX = (
    "Return a single PostgreSQL SELECT only.\n"
    "Use only tables: patients, encounters, conditions, observations, medication_requests, procedures.\n"
    "Use only parameter :patient_id. Prefer DISTINCT ON with ORDER BY for 'latest per X'; no CTEs or window functions.\n"
    "Do not mix GROUP BY with DISTINCT ON. If aggregation is needed (e.g., pairing BP), use GROUP BY + MAX(CASE...).\n"
    "Important: medication_requests has no rxnorm_code (use med_name only). Encounters has no location.\n"
    "If the question mentions 'blood pressure' or 'BP', return only systolic (8480-6) and diastolic (8462-4) results and prefer paired rows grouped by effective_datetime.\n"
    "\n"
    "Schema hints:\n"
    "  conditions(display, code, onset_datetime, abatement_datetime, encounter_id, patient_id, condition_id)\n"
    "  observations(display, loinc_code, value_num, value_unit, effective_datetime, encounter_id, patient_id, observation_id)\n"
    "  medication_requests(med_name, dose, route, start_datetime, end_datetime, refills, encounter_id, patient_id, med_request_id)\n"
    "  encounters(start_datetime, end_datetime, reason_text, class, encounter_id, patient_id)\n"
    "  procedures(display, code, performed_datetime, encounter_id, patient_id, procedure_id)\n"
    "Output only the raw SQL, no markdown fences."
)

def get_few_shots(query: str, k: int = 3):
    # uses your already-loaded FAISS vector store: `vs`
    docs_scores = vs.similarity_search_with_score(query, k=k)
    examples = [{"question": d.page_content, "sql": (d.metadata or {}).get("sql","")} for d, _ in docs_scores]
    return examples

def format_examples(examples):
    return "\n".join([f"Question: {ex['question']}\nSQL:\n{ex['sql']}\n" for ex in examples])

def build_prompt(user_question: str, k: int = 3) -> str:
    examples = get_few_shots(user_question, k=k)
    return f"{PREFIX}\n{format_examples(examples)}\nQuestion: {user_question}\nSQL:"


In [ ]:
# --- 5) SQL generation + a tiny safety check ---
import re

SELECT_ONLY = re.compile(r"^\s*select\b", re.IGNORECASE | re.DOTALL)

def clean_sql(text: str) -> str:
    s = text.strip()

    # strip a leading "SQL:" line if present
    if s.lower().startswith("sql:"):
        s = s[4:].strip()

    # strip fenced code blocks like ```sql ... ``` or ``` ... ```
    m = re.match(r"^```(?:\s*sql)?\s*([\s\S]*?)\s*```$", s, flags=re.IGNORECASE)
    if m:
        s = m.group(1).strip()

    # strip stray backticks if the model emitted them oddly
    if s.startswith("```") and "```" in s[3:]:
        s = s.split("```", 1)[1].rsplit("```", 1)[0].strip()

    # remove BOM or weird invisibles
    s = s.replace("\ufeff", "").replace("\u200b", "").strip()
    return s

def is_safe_select(text: str) -> bool:
    sql = clean_sql(text)

    # (optional) reject multiple statements (allow a single trailing semicolon)
    trimmed = sql.strip()
    if ";" in trimmed[:-1]:  # semicolon before the last char
        return False

    if not SELECT_ONLY.match(trimmed):
        return False

    banned = (" insert ", " update ", " delete ", " drop ", " alter ",
              " create ", " grant ", " revoke ", " truncate ")
    low = f" {trimmed.lower()} "  # pad with spaces to avoid substring accidents
    return not any(b in low for b in banned)

def generate_sql(user_question: str, k: int = 3, max_tokens: int = 400):
    prompt = build_prompt(user_question, k=k)
    resp = client.chat.completions.create(
        model=LLM_MODEL,
        temperature=0,
        messages=[
            {"role":"system","content":"You are a precise SQL generator for a patient portal."},
            {"role":"user","content": prompt}
        ],
        max_tokens=max_tokens,
    )
    sql = resp.choices[0].message.content.strip()
    return sql

# Demo
demo_q = "Which medications am I currently taking?"
sql = generate_sql(demo_q, k=3)
print(sql, "\n\nSAFE:", is_safe_select(sql))


SELECT DISTINCT ON (mr.patient_id, mr.med_name)
  mr.patient_id,
  mr.med_name AS medication,
  mr.dose,
  mr.route,
  mr.start_datetime,
  mr.end_datetime,
  mr.refills
FROM medication_requests mr
WHERE mr.patient_id = :patient_id
  AND (mr.end_datetime IS NULL OR mr.end_datetime >= NOW())
ORDER BY mr.patient_id,
         mr.med_name,
         COALESCE(mr.end_datetime, mr.start_datetime) DESC NULLS LAST; 

SAFE: True


# Load PostgreSQL EHR FHIR Database

In [ ]:
%%capture
!apt-get -y update
!apt-get -y install postgresql postgresql-contrib

!service postgresql start
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres createdb synthea_ehr

!echo "PostgreSQL installed, service started, user password set to 'postgres', and DB 'synthea_ehr' created."


In [ ]:
import subprocess
import os

# Database connection details
DB_NAME = "synthea_ehr"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"

# Path on Google Drive to the backup file
BACKUP_PATH = DEV_PATH + "/synthea_ehr_backup.sql"

def restore_database():
    """Restores the synthea_ehr database from a file on Google Drive."""
    try:
        print("Starting database restore...")

        # First, drop and re-create the database to ensure a clean state
        print("Dropping and re-creating the database for a clean restore...")
        env = os.environ.copy()
        env['PGPASSWORD'] = DB_PASSWORD

        # Command to drop the database
        drop_command = [
            'dropdb',
            '--host', DB_HOST,
            '--username', DB_USER,
            DB_NAME
        ]
        # This will fail if the DB doesn't exist, so we don't check for errors
        subprocess.run(drop_command, env=env, check=False, capture_output=True, text=True)

        # Command to create the database
        create_command = [
            'createdb',
            '--host', DB_HOST,
            '--username', DB_USER,
            DB_NAME
        ]
        subprocess.run(create_command, env=env, check=True, capture_output=True, text=True)
        print("Database re-created successfully.")

        # Use subprocess to run the psql command to restore the backup
        command = [
            'psql',
            '--host', DB_HOST,
            '--username', DB_USER,
            '--dbname', DB_NAME,
            '--file', BACKUP_PATH
        ]

        process = subprocess.run(command, env=env, check=True, capture_output=True, text=True)
        print("Database restore successful!")

    except FileNotFoundError:
        print("Error: psql or dropdb/createdb commands not found. Please ensure PostgreSQL client tools are installed.")
        print("You can try running: !apt-get update && !apt-get install -y postgresql-client")
    except subprocess.CalledProcessError as e:
        print("Error during restore process.")
        print(f"Subprocess returned non-zero exit code: {e.returncode}")
        print(f"STDOUT:\n{e.stdout}")
        print(f"STDERR:\n{e.stderr}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

restore_database()

Starting database restore...
Dropping and re-creating the database for a clean restore...
Database re-created successfully.
Database restore successful!


In [ ]:
# If needed once per runtime:
# !pip -q install psycopg2-binary

import re
import psycopg2
import psycopg2.extras

# Uses your existing globals:
# DB_NAME = "synthea_ehr"
# DB_USER = "postgres"
# DB_PASSWORD = "postgres"
# DB_HOST = "localhost"

# --- helpers ---
_SELECT_ONLY = re.compile(r"^\s*select\b", re.IGNORECASE | re.DOTALL)
_BANNED = (" insert ", " update ", " delete ", " drop ", " alter ",
           " create ", " grant ", " revoke ", " truncate ", " copy ", " do ")

# match :name (not preceded by another :)
_PARAM = re.compile(r'(?<!:):([a-zA-Z_]\w*)')

def _clean_sql(text: str) -> str:
    """Remove code fences / labels and invisible chars."""
    s = (text or "").strip()
    if s.lower().startswith("sql:"):
        s = s[4:].strip()
    m = re.match(r"^```(?:\s*sql)?\s*([\s\S]*?)\s*```$", s, flags=re.IGNORECASE)
    if m:
        s = m.group(1).strip()
    return s.replace("\ufeff","").replace("\u200b","").strip()

def _is_safe_select(sql: str) -> bool:
    """Single SELECT only; no DDL/DML keywords; no mid-string semicolons."""
    s = sql.strip()
    if ";" in s[:-1]:  # allow a single trailing semicolon only
        return False
    if not _SELECT_ONLY.match(s):
        return False
    low = f" {s.lower()} "
    return not any(b in low for b in _BANNED)

def _to_psycopg2_named(sql: str) -> str:
    """Convert :name placeholders to %(name)s for psycopg2."""
    return _PARAM.sub(r"%(\1)s", sql)

# --- main function ---
def execute_sql(sql: str, params: dict = None, timeout_ms: int = 5000):
    """
    Execute a single SELECT query safely and return rows as a list of dicts.

    Args:
      sql: SQL string (can use :param style placeholders, e.g., :patient_id)
      params: dict of parameters if placeholders are used (optional)
      timeout_ms: statement timeout in milliseconds (default 5000)

    Returns:
      List[Dict]: each row as a dict
    """
    raw = _clean_sql(sql)
    if not _is_safe_select(raw):
        raise ValueError("Blocked: SQL must be a single SELECT without DDL/DML.")

    q = _to_psycopg2_named(raw)
    params = params or {}

    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=5432,
        connect_timeout=5,
        options=f"-c statement_timeout={timeout_ms}"
    )
    try:
        with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
            cur.execute(q, params)
            return [dict(r) for r in cur.fetchall()]
    finally:
        conn.close()


In [ ]:
# Test

# Simple
rows = execute_sql("SELECT COUNT(*) AS n FROM patients;")
print(rows)


[{'n': 111}]


In [ ]:
rows = execute_sql("select patient_id from patients;")
print(rows[:3])

[{'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40'}, {'patient_id': '782001bc-f712-50ae-04f5-9a488f3ef4aa'}, {'patient_id': '80e7f50a-3e99-d5ac-cf97-f8a4b4f9e6c7'}]


In [ ]:
execute_sql("""
SELECT column_name
FROM information_schema.columns
WHERE table_schema='public' AND table_name='encounters'
ORDER BY 1;
""")

[{'column_name': 'class'},
 {'column_name': 'encounter_id'},
 {'column_name': 'end_datetime'},
 {'column_name': 'patient_id'},
 {'column_name': 'reason_text'},
 {'column_name': 'start_datetime'}]

In [ ]:
execute_sql("""
SELECT table_name, column_name
FROM information_schema.columns
WHERE table_schema='public'
ORDER BY 1;
""")

[{'table_name': 'conditions', 'column_name': 'display'},
 {'table_name': 'conditions', 'column_name': 'onset_datetime'},
 {'table_name': 'conditions', 'column_name': 'abatement_datetime'},
 {'table_name': 'conditions', 'column_name': 'encounter_id'},
 {'table_name': 'conditions', 'column_name': 'patient_id'},
 {'table_name': 'conditions', 'column_name': 'condition_id'},
 {'table_name': 'conditions', 'column_name': 'code'},
 {'table_name': 'encounters', 'column_name': 'class'},
 {'table_name': 'encounters', 'column_name': 'patient_id'},
 {'table_name': 'encounters', 'column_name': 'encounter_id'},
 {'table_name': 'encounters', 'column_name': 'reason_text'},
 {'table_name': 'encounters', 'column_name': 'end_datetime'},
 {'table_name': 'encounters', 'column_name': 'start_datetime'},
 {'table_name': 'medication_requests', 'column_name': 'dose'},
 {'table_name': 'medication_requests', 'column_name': 'start_datetime'},
 {'table_name': 'medication_requests', 'column_name': 'end_datetime'},
 {

In [ ]:
# With parameter (your canonical pattern)
q = """
SELECT DISTINCT ON (mr.patient_id, mr.med_name)
  mr.patient_id,
  mr.med_name AS medication,
  mr.dose,
  mr.route,
  mr.start_datetime,
  mr.end_datetime,
  mr.refills
FROM medication_requests mr
WHERE mr.patient_id = :patient_id
  AND (mr.end_datetime IS NULL OR mr.end_datetime >= NOW())
ORDER BY mr.patient_id,
         mr.med_name,
         COALESCE(mr.end_datetime, mr.start_datetime) DESC NULLS LAST;

"""
rows = execute_sql(q, {"patient_id": '8c8e1c9a-b310-43c6-33a7-ad11bad21c40'})
print(rows[:3])

[{'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40', 'medication': 'Acetaminophen 325 MG Oral Tablet', 'dose': None, 'route': None, 'start_datetime': None, 'end_datetime': None, 'refills': None}, {'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40', 'medication': 'Naproxen sodium 220 MG Oral Tablet', 'dose': None, 'route': None, 'start_datetime': None, 'end_datetime': None, 'refills': None}]


# Tie LLM output to return results from PostGre SQL database

In [ ]:
def answer_patient_question(user_question: str, patient_id: str, k: int = 3, max_tokens: int = 400 ):
    sql = generate_sql(user_question, k=k, max_tokens=max_tokens)
    #print("answer_patient_question: Generated SQL:\n", sql, "\n")
    rows = execute_sql(sql, {"patient_id": patient_id})
    #print("answer_patient_question: Executed SQL returned rows:\n", len(rows), "\n")
    return sql, rows


In [ ]:
execute_sql("select p.patient_id from patients p")

[{'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40'},
 {'patient_id': '782001bc-f712-50ae-04f5-9a488f3ef4aa'},
 {'patient_id': '80e7f50a-3e99-d5ac-cf97-f8a4b4f9e6c7'},
 {'patient_id': 'edc17058-55fb-08c7-12df-ece93a402e50'},
 {'patient_id': '9f9dbdcb-23a1-82cc-b7bc-e0e420a95bd1'},
 {'patient_id': 'be874504-c868-ebfd-9a77-df6b1e5ff6cc'},
 {'patient_id': '30e48e16-2df7-207e-7a3d-1650ef0c1ed8'},
 {'patient_id': '57b21dea-ff00-6c3e-92d9-91c7627f53b2'},
 {'patient_id': 'a3d34c1f-5421-e078-38ec-1498a5941dbe'},
 {'patient_id': 'e83fe1b3-f94f-5591-f851-1da300e24e99'},
 {'patient_id': 'e6705c33-7349-8b12-484d-3b1f93227178'},
 {'patient_id': '2da86d63-34ae-b887-ddff-8f6f1e6990f1'},
 {'patient_id': '04181caa-fcc1-c6c8-743e-a903eff368de'},
 {'patient_id': '20802592-1c31-7339-4c4c-2fe648e1a716'},
 {'patient_id': '406e8bad-81b5-7624-5b8a-4aeeb74028f5'},
 {'patient_id': 'a331b5bc-cbea-a205-a8bf-dbf3255ef36a'},
 {'patient_id': '641efcda-7397-4172-c6ac-8231342fa53e'},
 {'patient_id': 'e64918a6-528c-

In [ ]:
sql = (
    "SELECT e.patient_id, e.start_datetime, e.end_datetime, "
    "e.class AS encounter_class, e.reason_text AS reason "
    "FROM encounters e "
    "WHERE e.reason_text IS NOT NULL;"
)
execute_sql(sql)

[]

In [ ]:
import pandas as pd

patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = "List all my vital signs"

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
print("Generated SQL:\n", sql, "\n")
print("Rows:", len(rows))
if rows:
    display(pd.DataFrame(rows).head(10))

Generated SQL:
 SELECT
  o.patient_id,
  COALESCE(o.display, o.loinc_code) AS vital_name,
  o.value_num AS value,
  o.value_unit AS unit,
  o.effective_datetime
FROM observations AS o
WHERE o.patient_id = :patient_id
  AND (
    o.loinc_code IN ('8480-6','8462-4','8867-4','9279-1','8310-5','59408-5','29463-7','39156-5','8302-2')
    OR LOWER(o.display) IN (
      'systolic blood pressure','diastolic blood pressure',
      'heart rate','respiratory rate','body temperature',
      'oxygen saturation','body weight','bmi','body mass index','body height'
    )
  )
ORDER BY
  o.effective_datetime DESC NULLS LAST,
  COALESCE(o.display, o.loinc_code); 

Rows: 56


,patient_id,vital_name,value,unit,effective_datetime
0,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Height,183.4,cm,2025-04-21 14:19:34+00:00
1,0fca905f-391c-08d3-4b93-b53f69b9da53,Body mass index (BMI) [Ratio],28.35,kg/m2,2025-04-21 14:19:34+00:00
2,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Weight,95.4,kg,2025-04-21 14:19:34+00:00
3,0fca905f-391c-08d3-4b93-b53f69b9da53,Heart rate,69,/min,2025-04-21 14:19:34+00:00
4,0fca905f-391c-08d3-4b93-b53f69b9da53,Respiratory rate,14,/min,2025-04-21 14:19:34+00:00
5,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Height,183.4,cm,2024-04-15 14:19:34+00:00
6,0fca905f-391c-08d3-4b93-b53f69b9da53,Body mass index (BMI) [Ratio],28.26,kg/m2,2024-04-15 14:19:34+00:00
7,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Weight,95.1,kg,2024-04-15 14:19:34+00:00
8,0fca905f-391c-08d3-4b93-b53f69b9da53,Heart rate,83,/min,2024-04-15 14:19:34+00:00
9,0fca905f-391c-08d3-4b93-b53f69b9da53,Respiratory rate,15,/min,2024-04-15 14:19:34+00:00


# Post data frame, LLM genberates a nice summary

In [ ]:
import pandas as pd
from openai import OpenAI
import io

client = OpenAI()  # assumes OPENAI_API_KEY is set


In [ ]:
def df_to_csv_for_llm(df: pd.DataFrame, max_rows: int = 200, null_marker: str = "—") -> tuple[str, bool]:
    """
    Convert a DataFrame to CSV for the LLM.
    - Truncates to max_rows to keep prompts small.
    - Replaces NaNs with a visible marker (default "—").
    Returns (csv_text, truncated_flag).
    """
    truncated = False
    if len(df) > max_rows:
        df = df.head(max_rows).copy()
        truncated = True

    df = df.copy()
    df = df.fillna(null_marker)

    # keep column order stable
    csv_buf = io.StringIO()
    df.to_csv(csv_buf, index=False)
    return csv_buf.getvalue(), truncated

def summarize_df_with_llm(
    df: pd.DataFrame,
    patient_id: str,
    model: str = "gpt-4o-mini",
    max_rows: int = 200,
    null_marker: str = "—",
    max_tokens: int = 500
) -> str:
    """
    Ask the LLM to summarize a DataFrame.
    - Includes all visible (non-missing) values in its reasoning context,
      but the model will produce a concise natural-language summary (not a reprint of all rows).
    """
    if df is None or df.empty:
        return "No data found for the requested query."

    csv_text, truncated = df_to_csv_for_llm(df, max_rows=max_rows, null_marker=null_marker)
    columns_csv = ",".join(list(df.columns))

    user_prompt = f"""
You are a precise medical data summarizer. Use only the table below.
- Do not invent values or fields.
- Call out trends, counts, notable recency, and any obvious gaps (fields marked "{null_marker}").
- Keep it concise (2–5 sentences).
- If the table was truncated, say so and include how many rows were shown.

Patient: {patient_id}
Columns: {columns_csv}
Rows shown: {min(len(df), max_rows)}{' (truncated)' if truncated else ''}

CSV:
{csv_text}
""".strip()

    resp = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a precise, conservative medical data summarizer."},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=max_tokens,
    )
    return resp.choices[0].message.content.strip()


In [ ]:
# ##
# patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
# user_q = "What has been my highest weight"

# sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
# print("Generated SQL:\n", sql, "\n")
# print("Rows:", len(rows))
# # if rows:
# #     display(pd.DataFrame(rows).head(10))
# df = pd.DataFrame(rows)
# display(df.head(10))
# # 3) Summarize with the LLM
# summary = summarize_df_with_llm(df, patient_id="<REAL-PATIENT-ID>", model=LLM_MODEL)
# print(summary)

GroupingError: column "o.display" must appear in the GROUP BY clause or be used in an aggregate function
LINE 3:   COALESCE(o.display, o.loinc_code) AS vital_name,
                   ^


# Gradio UI

In [ ]:
# 0) Install Gradio (quiet)
!pip -q install gradio

# 1) Build a small adapter for the UI
import pandas as pd
import gradio as gr

def medintellagent_ui(patient_id: str, user_question: str, k: int = 3, max_tokens: int = 600):
    if not patient_id or not user_question:
        return "Please provide both patient_id and a question.", pd.DataFrame(), "—"
    try:
        # Generate SQL + fetch rows
        sql, rows = answer_patient_question(user_question, patient_id, k=k, max_tokens=max_tokens)
        df = pd.DataFrame(rows)
        if df.empty:
            summary = "No data found for this query (for this patient). Try another question or patient_id."
        else:
            summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
        return sql, df, summary
    except Exception as e:
        # Show errors cleanly in the UI (helpful during dev)
        return f"Error: {e}", pd.DataFrame(), "—"

# 2) Build the interface
with gr.Blocks(css="footer {visibility: hidden}") as demo:
    gr.Markdown("# MedIntellAgent — MVP UI")
    gr.Markdown("Enter a **Patient ID** and **Question**. You’ll get the generated SQL, raw DB rows, and an LLM summary.")

    with gr.Row():
        patient = gr.Textbox(label="Patient ID", placeholder="e.g., 0fca905f-391c-08d3-4b93-b53f69b9da53")
    question = gr.Textbox(label="Patient Question", lines=2,
                          placeholder="e.g., Which medications am I currently taking?")
    with gr.Row():
        k = gr.Slider(1, 5, value=3, step=1, label="Few-shot k")
        max_tokens = gr.Slider(100, 2000, value=600, step=50, label="LLM max_tokens")

    ask = gr.Button("Ask MedIntellAgent")

    sql_out = gr.Code(label="Generated SQL", language="sql")
    table_out = gr.Dataframe(label="Postgres Results", interactive=False)
    summary_out = gr.Markdown(label="LLM Summary")

    ask.click(medintellagent_ui,
              inputs=[patient, question, k, max_tokens],
              outputs=[sql_out, table_out, summary_out])

demo.launch(share=True)   # share=True if you want a temporary public link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://173689885f6262dae3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
